In [1]:
import matplotlib.pyplot as plt
%matplotlib inline

import torch
import torch.autograd as autograd
import numpy as np
import numpy.random as npr
import pandas as pd
from PIL import Image, ImageDraw

from src.region_proposal import RegionProposalNetwork, get_target_weights
from src.smooth_l1_loss import SmoothL1LossFunc
from src.generate_anchors import generate_anchors
from src.generate_anchor_data import AnchorDataGenerator

In [2]:
image = Image.open("data/JPEGImages/2010_006023.jpg")
image_a = np.asarray(image).astype(np.float32).transpose(2, 0, 1)
image_a = image_a[np.newaxis, :]
input = autograd.Variable(torch.Tensor(image_a))
im_w, im_h = autograd.Variable(torch.Tensor([image_a.shape[3]])), autograd.Variable(torch.Tensor([image_a.shape[2]]))

In [3]:
gt_df = pd.read_csv("data/Preprocess/Annotations/2010_006023.jpg.csv")
gt = gt_df.as_matrix()
gt_boxes = gt[:, 0:4]
gt_boxes = autograd.Variable(torch.Tensor(gt_boxes))

In [4]:
rpn = RegionProposalNetwork()
anchor_generator = AnchorDataGenerator()
log_softmax = torch.nn.LogSoftmax()
nll_loss = torch.nn.NLLLoss2d(weight=get_target_weights())

In [5]:
obj_scores, regressions = rpn(input)
labels, bbox_target, bbox_weights = anchor_generator(obj_scores, gt_boxes, im_w, im_h)

obj_scores = obj_scores.resize(1, 2, 9*obj_scores.size(2), obj_scores.size(3))
labels = labels.resize(1, labels.size(2), labels.size(3))

In [8]:
scores = log_softmax(obj_scores)
scores = torch.cat((scores, autograd.Variable(torch.zeros([1, 1, obj_scores.size(2), obj_scores.size(3)]))), 1)

In [15]:
loss = nll_loss(scores, labels)

In [16]:
loss

Variable containing:
 3.4918
[torch.FloatTensor of size 1]